In [ ]:
!nvidia-smi

Tue Jul 11 17:07:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -q langchain transformers sentence_transformers llama-index


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 115.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.0/580.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 7.0 MB/s eta 0:00:0

In [ ]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,GPTVectorStoreIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor, ServiceContext
import torch
from langchain.llms.base import LLM
from transformers import pipeline

In [ ]:

class customLLM(LLM):
    model_name = "google/flan-t5-Large"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]
    @property
    def _identifying_params(self):
        return {"name_of_model": self.model_name}
    @property
    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=customLLM())


In [ ]:

hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)


In [ ]:

from llama_index import SimpleDirectoryReader

bet = SimpleDirectoryReader('/content/drive/MyDrive/data/bet_health').load_data()
tumo = SimpleDirectoryReader('/content/drive/MyDrive/data/tumo_health').load_data()


In [ ]:
# set number of output tokens
num_output = 10000
# set maximum input size
max_input_size = 10000
# set maximum chunk overlap
max_chunk_overlap = 1


prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)


In [ ]:

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
# train for tumo
index_tumo = GPTVectorStoreIndex.from_documents(tumo, service_context=service_context, prompt_helper=prompt_helper)
# train for bet
index_bet = GPTVectorStoreIndex.from_documents(bet, service_context=service_context, prompt_helper=prompt_helper)

In [ ]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)

In [ ]:
# #test
# query_engine_tumo = index_tumo.as_query_engine()
# prompt = " what dental services does my insurance cover? "
# response = query_engine_tumo.query(prompt)
# print(response)

In [ ]:
# for tumo
query_engine_tumo = index_tumo.as_query_engine()
# for bet
query_engine_bet = index_bet.as_query_engine()

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
pip install deep_translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.1 MB/s eta 0:00:00


In [ ]:
from deep_translator import GoogleTranslator

In [ ]:

translator_am = GoogleTranslator(source='en', target='hy')
translator_en = GoogleTranslator(source='hy', target='en')

### Telegram bot

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
! pip install pytelegrambotapi aiogram


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 91.7 MB/s eta 0:00:00
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.12.0-py3-none-any.whl size=213956 sha256=ea6aaac54acea53e7e0244802665c96a67eeff9bfbdd74eabe336e8f778d0c22
  Stored in directory: /root/.cache/pip/wheels/7e/ba/82/f3ab5bc48525778633bccc741c0424677ed3435736221819f4
Successfully built pytelegrambotapi
  Attempting uninstall: Babel
    Found existing installation: Babel 2.12.1
    Uninstalling Babel-2.12.1:
      Successfully uninstalled Babel-2.12.1


In [ ]:
import telebot
from telebot import TeleBot
from telebot import types
from aiogram.types import ReplyKeyboardMarkup, ReplyKeyboardRemove, KeyboardButton

API_TOKEN = '6156036993:AAFP6oS_-bBJdLv1YDWUTto0PdCoZYaEihU'
bot = telebot.TeleBot(API_TOKEN)

selected_option = None  # Global variable to store the selected option

button_tumo = types.InlineKeyboardButton('Health_1', callback_data='Health_1')
button_bet = types.InlineKeyboardButton('Health_2', callback_data='Health_2')
keyboard = types.InlineKeyboardMarkup()
keyboard.add(button_tumo)
keyboard.add(button_bet)

@bot.message_handler(commands=['start'])
def start(message):
    global selected_option
    selected_option = None  # Reset the selected option
    bot.send_message(message.chat.id, 'Բարև, ես քեզ կարղ եմ օգնել ընտրել քեզ հարմար առողջապահական  ապահովաագրության տեսակ: Ընտրիր ցանկանանը', reply_markup=keyboard)

@bot.callback_query_handler(func=lambda call: call.data == 'Health_1')
def handle_health_1_callback(call):
    global selected_option
    selected_option = 'Health_1'
    bot.send_message(call.message.chat.id, 'Դուք ընտրել էք համար 1 ապահովագրության տեսակը, և ես պատրաստ եմ պատասխանել ձեր հարցերին')

@bot.callback_query_handler(func=lambda call: call.data == 'Health_2')
def handle_health_2_callback(call):
    global selected_option
    selected_option = 'Health_2'
    bot.send_message(call.message.chat.id, 'Դուք ընտրել էք համար 2 ապահովագրության տեսակը, և ես պատրասխանել ձեր հարցերին')

@bot.message_handler(func=lambda message: True)
def generate_response(message):
    global selected_option

    if selected_option == 'Health_1':
        input_text = translator_en.translate(message.text)

        query_engine_tumo = index_tumo.as_query_engine()
        test = "is the question related to health insurance plan?"
        response_test = query_engine_tumo.query(input_text + test)
        if str(response_test) == 'yes':
            response = query_engine_tumo.query(input_text)
            bot.reply_to(message, translator_am.translate(str(response)))
        else:
            response = 'Կներեք, չեմ կարող կատարել այդ խնդրանքը, քանի որ տրամադրված ՏԵՂԵԿԱՏՎՈՒԹՅՈՒՆԸ չի վերաբերում առողջության ապահովագրությանը:'
            bot.reply_to(message, response)

    if selected_option == 'Health_2':
        input_text = translator_en.translate(message.text)

        query_engine_bet = index_bet.as_query_engine()
        test = "is the question related to health insurance plan?"
        response_test = query_engine_bet.query(input_text + test)
        if str(response_test) == 'yes':
            response = query_engine_bet.query(input_text)
            bot.reply_to(message, translator_am.translate(str(response)))
        else:
            response = 'Կներեք, չեմ կարող կատարել այդ խնդրանքը, քանի որ տրամադրված ՏԵՂԵԿԱՏՎՈՒԹՅՈՒՆԸ չի վերաբերում առողջության ապահովագրությանը:'
            bot.reply_to(message, response)




In [ ]:
# and here we actually run it
bot.polling()

# Flask App

In [ ]:
chatbot_html = """
<style type="text/css">#log p { margin: 5px; font-family: sans-serif; }</style>
<div id="log"
     style="box-sizing: border-box;
            width: 600px;
            height: 32em;
            border: 1px grey solid;
            padding: 2px;
            overflow: scroll;">
</div>
<input type="text" id="typehere" placeholder="type here!"
       style="box-sizing: border-box;
              width: 600px;
              margin-top: 5px;">
<script>
function paraWithText(t) {
    let tn = document.createTextNode(t);
    let ptag = document.createElement('p');
    ptag.appendChild(tn);
    return ptag;
}
document.querySelector('#typehere').onchange = async function() {
    let inputField = document.querySelector('#typehere');
    let val = inputField.value;
    inputField.value = "";
    let resp = await getResp(val);
    let objDiv = document.getElementById("log");
    objDiv.appendChild(paraWithText('😀: ' + val));
    objDiv.appendChild(paraWithText('🤖: ' + resp));
    objDiv.scrollTop = objDiv.scrollHeight;
};
async function colabGetResp(val) {
    let resp = await google.colab.kernel.invokeFunction(
        'notebook.get_response', [val], {});
    return resp.data['application/json']['result'];
}
async function webGetResp(val) {
    let resp = await fetch("/response.json?sentence=" +
        encodeURIComponent(val));
    let data = await resp.json();
    return data['result'];
}
</script>
"""

In [ ]:
chatbot_html1 = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Chatbot</title>
    <style>
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }

        body {
            font-family: 'Nunito', sans-serif;
            font-weight: 400;
            font-size: 100%;
            background: #F1F1F1;
        }

        .container {
            display: flex;
            justify-content: flex-end;
            position: absolute;
            bottom: 30px;
            right: 30px;
        }

        .chatbox {
            position: relative;
            margin-right: 20px;
        }

        .chatbox__support {
            display: flex;
            flex-direction: column;
            background: #f9f9f9;
            height: 450px;
            width: 350px;
            box-shadow: 0px 0px 15px rgba(0, 0, 0, 0.1);
            border-top-left-radius: 20px;
            border-top-right-radius: 20px;
        }

        .chatbox__button {
            text-align: right;
        }

        .send__button {
            padding: 6px;
            background: transparent;
            border: none;
            outline: none;
            cursor: pointer;
        }






        .chatbox--active {
            transform: translateY(-40px);
            z-index: 123456;
            opacity: 1;
        }

        .chatbox__header {
            background: linear-gradient(93.12deg, #581B98 0.52%, #9C1DE7 100%);
            display: flex;
            flex-direction: row;
            align-items: center;
            justify-content: center;
            padding: 15px 20px;
            border-top-left-radius: 20px;
            border-top-right-radius: 20px;
            box-shadow: 0px 10px 15px rgba(0, 0, 0, 0.1);
        }

        .chatbox__image--header {
            margin-right: 10px;
        }

        .chatbox__heading--header {
            font-size: 1.2rem;
            color: white;
        }

        .chatbox__description--header {
            font-size: .9rem;
            color: white;
        }

        .chatbox__messages {
            padding: 0 20px;
            margin-top: auto;
            display: flex;
            overflow-y: scroll;
            flex-direction: column-reverse;
        }

        .messages__item {
            margin-top: 10px;
            background: #E0E0E0;
            padding: 8px 12px;
            max-width: 70%;
        }

        .messages__item--visitor,
        .messages__item--typing {
            border-top-left-radius: 20px;
            border-top-right-radius: 20px;
            border-bottom-right-radius: 20px;
        }

        .messages__item--operator {
            border-top-left-radius: 20px;
            border-top-right-radius: 20px;
            border-bottom-left-radius: 20px;
            background: #581B98;
            color: white;
        }

        .chatbox__footer {
            display: flex;
            flex-direction: row;
            align-items: center;
            justify-content: space-between;
            padding: 20px 20px;
            background: linear-gradient(268.91deg, #581B98 -2.14%, #9C1DE7 99.69%);
            box-shadow: 0px -10px 15px rgba(0, 0, 0, 0.1);
            border-bottom-right-radius: 10px;
            border-bottom-left-radius: 10px;
            margin-top: 20px;
        }

        .chatbox__footer input {
            width: 80%;
            border: none;
            padding: 10px 10px;
            border-radius: 30px;
            text-align: left;
        }

        .chatbox__send--footer {
            color: white;
        }

        .chatbox__button button,
        .chatbox__button button:focus,
        .chatbox__button button:visited {
            padding: 10px;
            background: white;
            border: none;
            outline: none;
            border-top-left-radius: 50px;
            border-top-right-radius: 50px;
            border-bottom-left-radius: 50px;
            box-shadow: 0px 10px 15px rgba(0, 0, 0, 0.1);
            cursor: pointer;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="chatbox">
            <div class="chatbox__support">
                <div class="chatbox__header">
                    <div class="chatbox__image--header">
                        <img src="https://img.icons8.com/color/48/000000/circled-user-female-skin-type-5--v1.png" alt="image">
                    </div>
                    <div class="chatbox__content--header">
                        <h4 class="chatbox__heading--header">Chat support</h4>
                        <p class="chatbox__description--header">Բարև, ես քեզ կարղ եմ օգնել ընտրել քեզ հարմար առողջապահական  ապահովաագրության տեսակ:</p>
                    </div>
                </div>
                <div class="chatbox__messages">
                    <div id="log"></div>
                </div>
                <div class="chatbox__footer">
                    <input type="text" id="typehere" placeholder="Write a message...">
                    <button class="chatbox__send--footer send__button">Send</button>
                </div>
            </div>
            <div class="chatbox__button">
                <button><img src="{{ url_for('static', filename='images/chatbox-icon.svg') }}" /></button>
            </div>
        </div>
    </div>

    <script>
        function paraWithText(t) {
            let tn = document.createTextNode(t);
            let ptag = document.createElement('p');
            ptag.appendChild(tn);
            return ptag;
        }

        document.querySelector('#typehere').onchange = async function() {
            let inputField = document.querySelector('#typehere');
            let val = inputField.value;
            inputField.value = "";
            let resp = await getResp(val);
            let objDiv = document.getElementById("log");
            objDiv.appendChild(paraWithText('😀: ' + val));
            objDiv.appendChild(paraWithText('🤖: ' + resp));
            objDiv.scrollTop = objDiv.scrollHeight;
        };

        async function colabGetResp(val) {
            let resp = await google.colab.kernel.invokeFunction('notebook.get_response', [val], {});
            return resp.data['application/json']['result'];
        }

        async function webGetResp(val) {
            let resp = await fetch("/response.json?sentence=" + encodeURIComponent(val));
            let data = await resp.json();
            return data['result'];
        }

    </script>

    <script>
        class Chatbox {
            constructor() {
                this.args = {
                    openButton: document.querySelector('.chatbox__button'),
                    chatBox: document.querySelector('.chatbox__support'),
                    sendButton: document.querySelector('.send__button')
                }

                this.state = false;
                this.messages = [];
            }

            display() {
                const { openButton, chatBox, sendButton } = this.args;

                openButton.addEventListener('click', () => this.toggleState(chatBox))

                sendButton.addEventListener('click', () => this.onSendButton(chatBox))

                const node = chatBox.querySelector('input');
                node.addEventListener("keyup", ({ key }) => {
                    if (key === "Enter") {
                        this.onSendButton(chatBox)
                    }
                })
            }

            toggleState(chatbox) {
                this.state = !this.state;

                // show or hides the box
                if (this.state) {
                    chatbox.classList.add('chatbox--active')
                } else {
                    chatbox.classList.remove('chatbox--active')
                }
            }

            onSendButton(chatbox) {
                var textField = chatbox.querySelector('input');
                let text1 = textField.value
                if (text1 === "") {
                    return;
                }

                let msg1 = { name: "User", message: text1 }
                this.messages.push(msg1);

                let resp = await webGetResp(text1);  // Using the existing webGetResp function

                let msg2 = { name: "Sam", message: resp };
                this.messages.push(msg2);
                this.updateChatText(chatbox)
                textField.value = ''
            }

            updateChatText(chatbox) {
                var html = '';
                this.messages.slice().reverse().forEach(function (item, index) {
                    if (item.name === "Sam") {
                        html += '<div class="messages__item messages__item--visitor">' + item.message + '</div>'
                    } else {
                        html += '<div class="messages__item messages__item--operator">' + item.message + '</div>'
                    }
                });

                const chatmessage = chatbox.querySelector('.chatbox__messages');
                chatmessage.innerHTML = html;
            }
        }


        const chatbox = new Chatbox();
        chatbox.display();
    </script>
</body>
</html>


"""

In [ ]:
!pip install flask_ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken 2SNtopC7mwSR8w1Vu5N4pERv38r_7ApCexK4x3pT6FnxiJsgj

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=4de0d06e17aa18b90560cca06c27a3d8d5f7ba253fd940c26522b5fbc5bf01a1
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://7l82sgtyfcd-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)


button_tumo = types.InlineKeyboardButton('Health_1', callback_data='Health_1')
button_bet = types.InlineKeyboardButton('Health_2', callback_data='Health_2')
keyboard = types.InlineKeyboardMarkup()
keyboard.add(button_tumo)
keyboard.add(button_bet)

@app.route("/response.json")
def response():
    sentence = request.args['sentence']
    if sentence == '/start':
      response = 'Բարև, ես քեզ կարղ եմ օգնել ընտրել քեզ հարմար առողջապահական  ապահովաագրության տեսակ:'
    # elif sentence == '1':

    # elif sentence == '2':

    else:
      response = query_engine_tumo.query(sentence)
    return jsonify({'result': str(response)})


@app.route("/")
def home():
    return chatbot_html1 + "<script>let getResp = webGetResp;</script>"

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://b272-34-29-10-239.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Jul/2023 17:39:13] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2023 17:39:13] "GET /{{%20url_for('static',%20filename='images/chatbox-icon.svg')%20}} HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2023 17:39:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2023 17:39:43] "GET /{{%20url_for('static',%20filename='images/chatbox-icon.svg')%20}} HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2023 17:39:43] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2023 17:40:00] "GET /response.json?sentence=hi HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2023 17:40:09] "GET /response.json?sentence=give%20a%20summary%20of%20health%20insurance%20plan HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2023 17:40:22] "GET /response.json?sentence=giva%20a%20summary HTTP/1.1" 200 -


In [ ]:
import telebot
from telebot import TeleBot
from telebot import types
from aiogram.types import ReplyKeyboardMarkup, ReplyKeyboardRemove, KeyboardButton

API_TOKEN = '6156036993:AAFP6oS_-bBJdLv1YDWUTto0PdCoZYaEihU'
bot = telebot.TeleBot(API_TOKEN)

selected_option = None  # Global variable to store the selected option

button_tumo = types.InlineKeyboardButton('Health_1', callback_data='Health_1')
button_bet = types.InlineKeyboardButton('Health_2', callback_data='Health_2')
keyboard = types.InlineKeyboardMarkup()
keyboard.add(button_tumo)
keyboard.add(button_bet)

@bot.message_handler(commands=['start'])
def start(message):
    global selected_option
    selected_option = None  # Reset the selected option
    bot.send_message(message.chat.id, 'Բարև, ես քեզ կարղ եմ օգնել ընտրել քեզ հարմար առողջապահական  ապահովաագրության տեսակ: Ընտրիր ցանկանանը', reply_markup=keyboard)

@bot.callback_query_handler(func=lambda call: call.data == 'Health_1')
def handle_health_1_callback(call):
    global selected_option
    selected_option = 'Health_1'
    bot.send_message(call.message.chat.id, 'Դուք ընտրել էք համար 1 ապահովագրության տեսակը, և ես պատրաստ եմ պատասխանել ձեր հարցերին')

@bot.callback_query_handler(func=lambda call: call.data == 'Health_2')
def handle_health_2_callback(call):
    global selected_option
    selected_option = 'Health_2'
    bot.send_message(call.message.chat.id, 'Դուք ընտրել էք համար 2 ապահովագրության տեսակը, և ես պատրասխանել ձեր հարցերին')

@bot.message_handler(func=lambda message: True)
def generate_response(message):
    global selected_option

    if selected_option == 'Health_1':
        input_text = translator_en.translate(message.text)

        query_engine_tumo = index_tumo.as_query_engine()
        test = "is the question related to health insurance plan?"
        response_test = query_engine_tumo.query(input_text + test)
        if str(response_test) == 'yes':
            response = query_engine_tumo.query(input_text)
            bot.reply_to(message, translator_am.translate(str(response)))
        else:
            response = 'Կներեք, չեմ կարող կատարել այդ խնդրանքը, քանի որ տրամադրված ՏԵՂԵԿԱՏՎՈՒԹՅՈՒՆԸ չի վերաբերում առողջության ապահովագրությանը:'
            bot.reply_to(message, response)

    if selected_option == 'Health_2':
        input_text = translator_en.translate(message.text)

        query_engine_bet = index_bet.as_query_engine()
        test = "is the question related to health insurance plan?"
        response_test = query_engine_bet.query(input_text + test)
        if str(response_test) == 'yes':
            response = query_engine_bet.query(input_text)
            bot.reply_to(message, translator_am.translate(str(response)))
        else:
            response = 'Կներեք, չեմ կարող կատարել այդ խնդրանքը, քանի որ տրամադրված ՏԵՂԵԿԱՏՎՈՒԹՅՈՒՆԸ չի վերաբերում առողջության ապահովագրությանը:'
            bot.reply_to(message, response)




In [ ]:
'Sorry, I’m afraid I cannot fulfill that request as the provided CONTEXT INFORMATION does not relate to health insurance'

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# pip install deep_translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
# import os
# import json
# from deep_translator import GoogleTranslator

In [ ]:
#  translator = GoogleTranslator(source='auto', target='en')

In [ ]:
# print(translator.translate("Կարո՞ղ եմ մեջքիս ռենտգեն անել:"))

Can I have an X-ray of my back?


In [ ]:
# query_engine = index.as_query_engine()

# translator_to_en = GoogleTranslator(source='auto', target='en')
# prompt_hy = "Կարո՞ղ եմ մեջքիս ռենտգեն անել:"
# print(prompt_hy)
# prompt_en = translator_to_en.translate(prompt_hy)
# print(prompt_en)

# response_en = query_engine.query(prompt_en)
# print(response_en)
# translator_to_hy = GoogleTranslator(source='auto', target='hy')
# response_hy = translator_to_hy.translate(str(response_en))
# print(response_hy)

Կարո՞ղ եմ մեջքիս ռենտգեն անել:
Can I have an X-ray of my back?
yes
այո
